# Facebook爬蟲

## 目標
- 自己客戶釋放離開訊息
- 競業客戶釋放離開訊息
- 來搶188的客戶其實都是潛在客戶
- 去競業搶促案的客戶價值更高
- 客戶價值：競業流失 > 台星流失 > 競業促案 > 台星促案 > 
- 加入時間維度，一直參加活動表示有強烈意願
- 貼文的分類，促案/新知。。。
## To Do List
- 先在外層抓貼文資訊，若按讚、留言數量相同，則不用重複再抓留言內容
- 偵測reply的方式需要調整，只偵測reply會在hide跟more replies中陷入無窮迴圈
- 每篇貼文需有上限的抓取時間，可能是貼文數*3秒，超過總時間則需要跳出

In [1]:
import pandas as pd
import re, time, requests, datetime, gc
from selenium import webdriver
from bs4 import BeautifulSoup

# 定義函數供後續使用

在這裡我們先定義一個函數，希望把網頁中各篇貼文的連結都找出來!
ulr放我們要爬的Facebook網址，n是稍後要送出幾次滾動網頁到底部的命令，藉以加載更多資料。

In [2]:
def FindLinks(url, n):
    Links = []
    driver.get(url)
    for i in range(n):
        time.sleep(1)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    # 這裡會跳出要我們登入的大畫面，找到「稍後再說」的按鈕並點擊
    driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
    soup = BeautifulSoup(driver.page_source)
    posts = soup.findAll('div', {'class':'clearfix y_c3pyo2ta3'})
    for i in posts:
        Links.append('https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0])
    return Links

定義一個展開所有留言的函數，透過while迴圈反覆搜尋與點擊「看更多留言」、「看更多回覆」與「看完整貼文內容」等按鈕。
在過程中會出現請我們登入或註冊的彈跳視窗，但我們不確定到底什麼時候會跳出，因此需要在過程中反覆偵測是有出現這個彈窗，若有就點擊「Not Now」

In [3]:
# 展開url中的所有留言 與 留言的留言
def expand(url):
    driver.get(url)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    try:
        driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
        time.sleep(1)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        driver.find_element_by_id('expanding_cta_close_button').click() 
    except:
        time.sleep(1)
    # 偵測是否有「更多留言」（第一層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))>0:
        for i in driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            try:
                driver.find_element_by_id('expanding_cta_close_button').click()
            except:
                time.sleep(0.5)            
            try:
                i.click()
            except:
                time.sleep(0.5)   
    # 偵測是否有「更多留言的留言」（第二層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))>0:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        for i in driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            try:
                driver.find_element_by_id('expanding_cta_close_button').click()
            except:
                time.sleep(0.5)
            try:
                i.click()
            except:
                time.sleep(0.5)
    # 偵測是否有「更多留言的留言」（第二層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))>0:
        for i in driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            try:
                i.click()
            except:
                time.sleep(0.5)

透過逐層搜索的方式，逐步定位我們要找的資訊
在這個環節需要反覆透過Chrome的功能比對資料，需要花一些心力進行比對
另外在這部分也使用了大量個try-except，原因是許多資料是有內容才會出現。例如並非每天貼人都會收到「哈哈」、「生氣」的心情。

In [4]:
# 文章內容與互動摘要
def CrawlPost(soup):
    time.sleep(1)
    # po文區塊
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # po文人資訊區塊
    PosterInfo = userContent.find('div', {'class':'l_c3pyo2v0u i_c3pynyi2f clearfix'})
    # 互動摘要區(讚、留言與分享)
    feedback = soup.find('form', {'class':'commentable_item collapsed_comments'})
    # 名稱
    Name = PosterInfo.find('img').attrs['aria-label']
    # ID
    ID = PosterInfo.find('a', {'class':'_5pb8 o_c3pynyi2g _8o _8s lfloat _ohe'}).attrs['href'].split('/?',2)[0].split('/',-1)[-1]
    # 網址
    Link = driver.current_url
    # 發文時間
    try:
        Time = PosterInfo.find('abbr').attrs['title']
    except:
        Time = PosterInfo.find('div', {'class':'_1atc fsm fwn fcg'}).text
    # 文章內容
    try:
        Content = userContent.find('div', {'class':'_5pbx userContent _3576'}).text
    except:
        Content = ""
    # Like
    try:
        Like = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_LIKE'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Like = '0' 
    # Love
    try:
        Love = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_LOVE'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Love = '0' 
    # Haha
    try:
        Haha = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_HAHA'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Haha = '0' 
    # Wow
    try:
        Wow = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_WOW'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Wow = '0' 
    # Sad
    try:
        Sad = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_SORRY'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Sad = '0' 
    # Angry
    try:
        Angry = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_ANGER'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Angry = '0'
    # 留言
    try:
        Comments = feedback.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
    except:
        Comments = '0' 
    # 分享
    try:
        Share = feedback.find('span', {'class':'_355t _4vn2'}).text.split(' ',2)[0]
    except:
        Share = '0' 
    Updatetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return pd.DataFrame(
        data = [{'Name':Name,
                 'ID':ID,
                 'Link':Link,
                 'Time':Time,
                 'Content':Content,
                 'Like':Like,
                 'Love':Love,
                 'Haha':Haha,
                 'Wow':Wow,
                 'Sad':Sad,
                 'Angry':Angry,
                 'Comments':Comments,
                 'Share':Share,
                 'Updatetime':Updatetime}],
        columns = ['Name', 'ID', 'Time', 'Content', 'Like', 'Love', 'Haha', 'Wow', 'Sad', 'Angry', 'Comments', 'Share', 'Link', 'Updatetime'])

這邊要留意雖然都是粉絲留言，但實際上分成「回應貼文的留言」與「回應留言的留言」。
函數中的第一個迴圈是用來抓「回應貼文的留言」，第二個則是抓「回應留言的留言」。讀者可以自行比較一下兩個迴圈中不同的地方。

In [5]:
# 留言
def CrawlComment(soup):
    time.sleep(1)
    Comments = pd.DataFrame()
    # po文區塊
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # 用戶留言區
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # 回應貼文的留言
    for i in userContent.findAll('div', {'data-testid':'UFI2Comment/root_depth_0'}):
        try:
            Content = i.find('span', {'dir':'ltr'}).text
        except:
            Content = 'sticker'
        Comment = pd.DataFrame(data = [{'ID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                 'Name':i.find('img').attrs['alt'],
                                 'Time':i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'],
                                 'Content':Content,
                                 'Link':driver.current_url}],
                        columns = ['ID', 'Name', 'Time', 'Content', 'Link'])
        Comments = pd.concat([Comments, Comment], ignore_index=True)
    
    # 回應留言的留言
    for i in userContent.findAll('div', {'data-testid':'UFI2Comment/root_depth_1'}):
        try:
            Content = i.find('span', {'dir':'ltr'}).text
        except:
            Content = 'sticker'
        Comment = pd.DataFrame(data = [{'ID':i.find('a', {'class':' _3mf5 _3mg1'}).attrs['data-hovercard'].split('id=',2)[1],
                                 'Name':i.find('img').attrs['alt'],
                                 'Time':i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'],
                                 'Content':Content,
                                 'Link':driver.current_url}],
                        columns = ['ID', 'Name', 'Time', 'Content', 'Link'])
        Comments = pd.concat([Comments, Comment], ignore_index=True)
    Comments['Updatetime'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return Comments

# 組合應用
組合以上函數，針對逐一爬取List中的貼文，依據貼文的類型做後續處理

- 新貼文：爬取貼文與留言
- 舊貼文但有新留言：更新貼文摘要並爬取留言
- 舊貼文且無新留言：pass
- 其他：查看情況

In [6]:
#def CrawlWall(url, n):
#    Links = FindLinks(url = url, n = n)
#    # 抓下來所有留言
#    for i in Links:
#        print('Dealing with: ' + i)
#        try:
#            driver.get(i)
#            time.sleep(1)
#            soup = BeautifulSoup(driver.page_source)
#            # 檢視文章摘要，分成全新文章、舊文章但有新留言、舊文章也無新留言
#            nPost = CrawlPost(soup)
#            compare = pd.merge(left = nPost.loc[:,['Link', 'Comments']] ,
#                               right = Posts.loc[:,['Link', 'Comments']],
#                               how = 'inner',
#                               on = ['Link'],
#                               indicator=True,
#                               suffixes=('_n', '_o'))
#            # 新貼文直接爬留言
#            if len(compare) == 0:
#                print('Type: ' + 'New Post')
#                expand(i)
#                soup = BeautifulSoup(driver.page_source)
#                nComments = CrawlComment(soup)
#                # 舊文章但有新留言，僅保留新的留言(drop掉已經在資料庫的資料)
#            elif compare.Comments_n.values > compare.Comments_o.values:
#                # 開始爬留言並將文章摘要匯入
#                print('Type: ' + 'Old Post with New Comments')
#                expand(i)
#                soup = BeautifulSoup(driver.page_source)
#                nComments = CrawlComment(soup)
#                nComments = pd.merge(left = nComments,
#                                     right = Comments,
#                                     how = 'left',
#                                     on = ['ID', 'Name', 'Content', 'Link'],
#                                     suffixes=('', '_o'),
#                                     indicator=True)
#                nComments = nComments.loc[nComments._merge == 'left_only',:]
#                nComments = nComments.drop(['Time_o', 'Updatetime_o', '_merge'], axis=1)
#            # 舊文章也無新留言，pass過去
#            elif compare.Comments_n.values == compare.Comments_o.values:
#                print('Type: ' + 'Old Post with Old Comments')
#                nComments = pd.DataFrame()
#            # 其他狀況，請釐清問題
#            else:
#                print('Type: ' + 'Uncertain condiction! Plz Check this website:' + '...')
#                nComments = pd.DataFrame()        
#        
#            # post的資料直接concat在最下面，並duplicate掉，藉以更新互動摘要
#            Posts = pd.concat([Posts, nPost], ignore_index=True)
#            Posts = Posts.sort_values(by = 'Updatetime', ascending = False)
#            Posts = Posts.drop_duplicates(subset ="Link",
#                                          keep= 'first',
#                                          inplace = False) 
#            Comments = pd.concat([Comments, nComments], ignore_index=True, sort=False)
#            print('Result: Succed!')
#            print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
#            print('----------------------------------')
#            print('\n')
#        except:
#            print('Result: Failed!')
#            print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
#            print('----------------------------------')
#            print('\n')
#        Result = [Posts, Comments]
#        return Result

In [7]:
# https://www.facebook.com/cathayunitedbank/
# https://www.facebook.com/tstartel/
# https://www.facebook.com/chtmobile/
# https://www.facebook.com/taiwanmobile/
# https://www.facebook.com/fareastone/
# https://www.facebook.com/Aptg.tw/
driver = webdriver.Chrome()
driver.get('https://www.facebook.com/')
time.sleep(1.5)
driver.find_element_by_xpath('//a[@class="_sv4"]').click()

In [11]:
# 載入先前資料
Posts = pd.read_csv('C:/Users/TL_Yu/Desktop/Posts.csv',
                    skiprows=[0],
                    names = ['Name', 'ID', 'Time', 'Content', 'Like', 'Love', 'Haha', 'Wow', 'Sad', 'Angry', 'Comments', 'Share', 'Link', 'Updatetime'],
                    dtype='str')
Comments = pd.read_csv('C:/Users/TL_Yu/Desktop/Comments.csv',
                       skiprows=[0],
                       names=['Content', 'ID', 'Link', 'Name', 'Time', 'Updatetime'],
                       dtype='str')

#Posts = pd.DataFrame()
#Comments = pd.DataFrame()

## 台星

In [12]:
url = 'https://www.facebook.com/tstartel/'
Links = FindLinks(url = url, n = 15)
# 抓下來所有留言
for i in Links:
    print('Dealing with: ' + i)
    try:
        driver.get(i)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source)
        # 檢視文章摘要，分成全新文章、舊文章但有新留言、舊文章也無新留言
        nPost = CrawlPost(soup)
        compare = pd.merge(left = nPost.loc[:,['Link', 'Comments']] ,
                           right = Posts.loc[:,['Link', 'Comments']],
                           how = 'inner',
                           on = ['Link'],
                           indicator=True,
                           suffixes=('_n', '_o'))
        # 新貼文直接爬留言
        if len(compare) == 0:
            print('Type: ' + 'New Post')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
        # 舊文章但有新留言，僅保留新的留言(drop掉已經在資料庫的資料)
        elif compare.Comments_n.values > compare.Comments_o.values:
            # 開始爬留言並將文章摘要匯入
            print('Type: ' + 'Old Post with New Comments')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
            nComments = pd.merge(left = nComments,
                                 right = Comments,
                                 how = 'left',
                                 on = ['ID', 'Name', 'Content', 'Link'],
                                 suffixes=('', '_o'),
                                 indicator=True)
            nComments = nComments.loc[nComments._merge == 'left_only',:]
            nComments = nComments.drop(['Time_o', 'Updatetime_o', '_merge'], axis=1)
        # 舊文章也無新留言，pass過去
        elif compare.Comments_n.values == compare.Comments_o.values:
            print('Type: ' + 'Old Post with Old Comments')
            nComments = pd.DataFrame()
        # 其他狀況，請釐清問題
        else:
            print('Type: ' + 'Uncertain condiction! Plz Check this website:' + '...')
            nComments = pd.DataFrame()        
        
        # post的資料直接concat在最下面，並duplicate掉，藉以更新互動摘要
        Posts = pd.concat([Posts, nPost], ignore_index=True)
        Posts = Posts.sort_values(by = 'Updatetime', ascending = False)
        Posts = Posts.drop_duplicates(subset ="Link",
                                      keep= 'first',
                                      inplace = False) 
        Comments = pd.concat([Comments, nComments], ignore_index=True, sort=False)
        print('Result: Succed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
    except:
        print('Result: Failed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
Posts.to_csv('C:/Users/TL_Yu/Desktop/Posts.csv',index = False)
Comments.to_csv('C:/Users/TL_Yu/Desktop/Comments.csv',index = False)

Dealing with: https://www.facebook.com/tstartel/videos/321955001812559/
Type: New Post
Result: Succed!
Time Log: 2019-05-15 22:14:47
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/posts/3269351643090860
Type: Uncertain condiction! Plz Check this website:...
Result: Succed!
Time Log: 2019-05-15 22:14:51
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/posts/3267141889978502
Type: Old Post with New Comments
Result: Succed!
Time Log: 2019-05-15 22:15:17
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3259789660713725/
Type: Old Post with New Comments
Result: Succed!
Time Log: 2019-05-15 22:15:32
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3264107533615271/
Type: Old Post with Old Comments
Result:

Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:18:50
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/photos/a.854369967922385/3197740693585289/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:18:53
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/posts/3194840400541985
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:18:57
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/photos/a.2482674201758612/3192347447457947/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:19:01
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/videos/1285036481654086/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:19:05
----------------------------------

In [13]:
gc.collect()

12858

## 中華

In [14]:
url = 'https://www.facebook.com/chtmobile/'
Links = FindLinks(url = url, n = 15)
# 抓下來所有留言
for i in Links:
    print('Dealing with: ' + i)
    try:
        driver.get(i)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source)
        # 檢視文章摘要，分成全新文章、舊文章但有新留言、舊文章也無新留言
        nPost = CrawlPost(soup)
        compare = pd.merge(left = nPost.loc[:,['Link', 'Comments']] ,
                           right = Posts.loc[:,['Link', 'Comments']],
                           how = 'inner',
                           on = ['Link'],
                           indicator=True,
                           suffixes=('_n', '_o'))
        # 新貼文直接爬留言
        if len(compare) == 0:
            print('Type: ' + 'New Post')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
        # 舊文章但有新留言，僅保留新的留言(drop掉已經在資料庫的資料)
        elif compare.Comments_n.values > compare.Comments_o.values:
            # 開始爬留言並將文章摘要匯入
            print('Type: ' + 'Old Post with New Comments')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
            nComments = pd.merge(left = nComments,
                                 right = Comments,
                                 how = 'left',
                                 on = ['ID', 'Name', 'Content', 'Link'],
                                 suffixes=('', '_o'),
                                 indicator=True)
            nComments = nComments.loc[nComments._merge == 'left_only',:]
            nComments = nComments.drop(['Time_o', 'Updatetime_o', '_merge'], axis=1)
        # 舊文章也無新留言，pass過去
        elif compare.Comments_n.values == compare.Comments_o.values:
            print('Type: ' + 'Old Post with Old Comments')
            nComments = pd.DataFrame()
        # 其他狀況，請釐清問題
        else:
            print('Type: ' + 'Uncertain condiction! Plz Check this website:' + '...')
            nComments = pd.DataFrame()        
        
        # post的資料直接concat在最下面，並duplicate掉，藉以更新互動摘要
        Posts = pd.concat([Posts, nPost], ignore_index=True)
        Posts = Posts.sort_values(by = 'Updatetime', ascending = False)
        Posts = Posts.drop_duplicates(subset ="Link",
                                      keep= 'first',
                                      inplace = False) 
        Comments = pd.concat([Comments, nComments], ignore_index=True, sort=False)
        print('Result: Succed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
    except:
        print('Result: Failed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
Posts.to_csv('C:/Users/TL_Yu/Desktop/Posts.csv',index = False)
Comments.to_csv('C:/Users/TL_Yu/Desktop/Comments.csv',index = False)

Dealing with: https://www.facebook.com/chtmobile/videos/2758708034143708/
Type: Old Post with New Comments
Result: Succed!
Time Log: 2019-05-15 22:21:21
----------------------------------------------------------


Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2715269541836004/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:21:24
----------------------------------------------------------


Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2707938675902424/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:21:28
----------------------------------------------------------


Dealing with: https://www.facebook.com/chtmobile/posts/2703335766362715
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:21:31
----------------------------------------------------------


Dealing with: https://www.facebook.com/chtmobile/posts/2695733070456318
Type: Old Post with Old Comments
Resul

Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:23:21
----------------------------------------------------------


Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2624215924274700/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:23:24
----------------------------------------------------------


Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2613377002025259/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:23:28
----------------------------------------------------------


Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2613255065370786/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:23:31
----------------------------------------------------------


Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2614263715269921/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22

In [15]:
gc.collect()

66960

## 台哥大

In [16]:
url = 'https://www.facebook.com/taiwanmobile/'
Links = FindLinks(url = url, n = 15)
# 抓下來所有留言
for i in Links:
    print('Dealing with: ' + i)
    try:
        driver.get(i)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source)
        # 檢視文章摘要，分成全新文章、舊文章但有新留言、舊文章也無新留言
        nPost = CrawlPost(soup)
        compare = pd.merge(left = nPost.loc[:,['Link', 'Comments']] ,
                           right = Posts.loc[:,['Link', 'Comments']],
                           how = 'inner',
                           on = ['Link'],
                           indicator=True,
                           suffixes=('_n', '_o'))
        # 新貼文直接爬留言
        if len(compare) == 0:
            print('Type: ' + 'New Post')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
        # 舊文章但有新留言，僅保留新的留言(drop掉已經在資料庫的資料)
        elif compare.Comments_n.values > compare.Comments_o.values:
            # 開始爬留言並將文章摘要匯入
            print('Type: ' + 'Old Post with New Comments')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
            nComments = pd.merge(left = nComments,
                                 right = Comments,
                                 how = 'left',
                                 on = ['ID', 'Name', 'Content', 'Link'],
                                 suffixes=('', '_o'),
                                 indicator=True)
            nComments = nComments.loc[nComments._merge == 'left_only',:]
            nComments = nComments.drop(['Time_o', 'Updatetime_o', '_merge'], axis=1)
        # 舊文章也無新留言，pass過去
        elif compare.Comments_n.values == compare.Comments_o.values:
            print('Type: ' + 'Old Post with Old Comments')
            nComments = pd.DataFrame()
        # 其他狀況，請釐清問題
        else:
            print('Type: ' + 'Uncertain condiction! Plz Check this website:' + '...')
            nComments = pd.DataFrame()        
        
        # post的資料直接concat在最下面，並duplicate掉，藉以更新互動摘要
        Posts = pd.concat([Posts, nPost], ignore_index=True)
        Posts = Posts.sort_values(by = 'Updatetime', ascending = False)
        Posts = Posts.drop_duplicates(subset ="Link",
                                      keep= 'first',
                                      inplace = False) 
        Comments = pd.concat([Comments, nComments], ignore_index=True, sort=False)
        print('Result: Succed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
    except:
        print('Result: Failed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
Posts.to_csv('C:/Users/TL_Yu/Desktop/Posts.csv',index = False)
Comments.to_csv('C:/Users/TL_Yu/Desktop/Comments.csv',index = False)

Dealing with: https://www.facebook.com/taiwanmobile/photos/a.1448377238716046/2473289892891437/
Type: Old Post with New Comments
Result: Succed!
Time Log: 2019-05-15 22:26:48
----------------------------------------------------------


Dealing with: https://www.facebook.com/taiwanmobile/photos/a.1448377238716046/2474118609475232/
Type: New Post
Result: Succed!
Time Log: 2019-05-15 22:26:55
----------------------------------------------------------


Dealing with: https://www.facebook.com/taiwanmobile/posts/2474400126113747
Type: New Post
Result: Succed!
Time Log: 2019-05-15 22:27:03
----------------------------------------------------------


Dealing with: https://www.facebook.com/taiwanmobile/videos/606775269838497/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:27:07
----------------------------------------------------------


Dealing with: https://www.facebook.com/taiwanmobile/posts/2471318653088561
Type: Old Post with New Comments
Result: Failed!
Time Log:

Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:29:32
----------------------------------------------------------


Dealing with: https://www.facebook.com/taiwanmobile/posts/2460245490862544
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:29:35
----------------------------------------------------------


Dealing with: https://www.facebook.com/taiwanmobile/posts/2460199744200452
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:29:39
----------------------------------------------------------


Dealing with: https://www.facebook.com/taiwanmobile/photos/a.1448377238716046/2459996730887420/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:29:43
----------------------------------------------------------


Dealing with: https://www.facebook.com/taiwanmobile/posts/2459393190947774
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:29:47
----------------------------------------

In [17]:
gc.collect()

87006

## 遠傳

In [18]:
url = 'https://www.facebook.com/fareastone/'
Links = FindLinks(url = url, n = 15)
# 抓下來所有留言
for i in Links:
    print('Dealing with: ' + i)
    try:
        driver.get(i)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source)
        # 檢視文章摘要，分成全新文章、舊文章但有新留言、舊文章也無新留言
        nPost = CrawlPost(soup)
        compare = pd.merge(left = nPost.loc[:,['Link', 'Comments']] ,
                           right = Posts.loc[:,['Link', 'Comments']],
                           how = 'inner',
                           on = ['Link'],
                           indicator=True,
                           suffixes=('_n', '_o'))
        # 新貼文直接爬留言
        if len(compare) == 0:
            print('Type: ' + 'New Post')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
        # 舊文章但有新留言，僅保留新的留言(drop掉已經在資料庫的資料)
        elif compare.Comments_n.values > compare.Comments_o.values:
            # 開始爬留言並將文章摘要匯入
            print('Type: ' + 'Old Post with New Comments')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
            nComments = pd.merge(left = nComments,
                                 right = Comments,
                                 how = 'left',
                                 on = ['ID', 'Name', 'Content', 'Link'],
                                 suffixes=('', '_o'),
                                 indicator=True)
            nComments = nComments.loc[nComments._merge == 'left_only',:]
            nComments = nComments.drop(['Time_o', 'Updatetime_o', '_merge'], axis=1)
        # 舊文章也無新留言，pass過去
        elif compare.Comments_n.values == compare.Comments_o.values:
            print('Type: ' + 'Old Post with Old Comments')
            nComments = pd.DataFrame()
        # 其他狀況，請釐清問題
        else:
            print('Type: ' + 'Uncertain condiction! Plz Check this website:' + '...')
            nComments = pd.DataFrame()        
        
        # post的資料直接concat在最下面，並duplicate掉，藉以更新互動摘要
        Posts = pd.concat([Posts, nPost], ignore_index=True)
        Posts = Posts.sort_values(by = 'Updatetime', ascending = False)
        Posts = Posts.drop_duplicates(subset ="Link",
                                      keep= 'first',
                                      inplace = False) 
        Comments = pd.concat([Comments, nComments], ignore_index=True, sort=False)
        print('Result: Succed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
    except:
        print('Result: Failed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
Posts.to_csv('C:/Users/TL_Yu/Desktop/Posts.csv',index = False)
Comments.to_csv('C:/Users/TL_Yu/Desktop/Comments.csv',index = False)

Dealing with: https://www.facebook.com/fareastone/photos/a.163720023641308/2536222723057681/
Type: New Post
Result: Succed!
Time Log: 2019-05-15 22:33:22
----------------------------------------------------------


Dealing with: https://www.facebook.com/fareastone/photos/a.163720023641308/2535914766421810/
Type: New Post
Result: Succed!
Time Log: 2019-05-15 22:33:37
----------------------------------------------------------


Dealing with: https://www.facebook.com/fareastone/videos/328924844668284/
Type: Old Post with New Comments
Result: Succed!
Time Log: 2019-05-15 22:33:48
----------------------------------------------------------


Dealing with: https://www.facebook.com/fareastone/posts/2527683350578285
Type: Old Post with New Comments
Result: Succed!
Time Log: 2019-05-15 22:34:02
----------------------------------------------------------


Dealing with: https://www.facebook.com/fareastone/photos/a.163720023641308/2527650693914884/
Type: Old Post with Old Comments
Result: Succed!
T

Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:36:57
----------------------------------------------------------


Dealing with: https://www.facebook.com/fareastone/posts/2500004880012799
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:37:01
----------------------------------------------------------


Dealing with: https://www.facebook.com/fareastone/photos/a.163720023641308/2498842046795749/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:37:05
----------------------------------------------------------


Dealing with: https://www.facebook.com/fareastone/posts/2498841476795806
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:37:08
----------------------------------------------------------


Dealing with: https://www.facebook.com/fareastone/photos/a.163720023641308/2490071891006098/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:37:12
-----------------------------

In [19]:
gc.collect()

26392

## 亞太

In [20]:
url = 'https://www.facebook.com/Aptg.tw/'
Links = FindLinks(url = url, n = 30)
# 抓下來所有留言
for i in Links:
    print('Dealing with: ' + i)
    try:
        driver.get(i)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source)
        # 檢視文章摘要，分成全新文章、舊文章但有新留言、舊文章也無新留言
        nPost = CrawlPost(soup)
        compare = pd.merge(left = nPost.loc[:,['Link', 'Comments']] ,
                           right = Posts.loc[:,['Link', 'Comments']],
                           how = 'inner',
                           on = ['Link'],
                           indicator=True,
                           suffixes=('_n', '_o'))
        # 新貼文直接爬留言
        if len(compare) == 0:
            print('Type: ' + 'New Post')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
        # 舊文章但有新留言，僅保留新的留言(drop掉已經在資料庫的資料)
        elif compare.Comments_n.values > compare.Comments_o.values:
            # 開始爬留言並將文章摘要匯入
            print('Type: ' + 'Old Post with New Comments')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
            nComments = pd.merge(left = nComments,
                                 right = Comments,
                                 how = 'left',
                                 on = ['ID', 'Name', 'Content', 'Link'],
                                 suffixes=('', '_o'),
                                 indicator=True)
            nComments = nComments.loc[nComments._merge == 'left_only',:]
            nComments = nComments.drop(['Time_o', 'Updatetime_o', '_merge'], axis=1)
        # 舊文章也無新留言，pass過去
        elif compare.Comments_n.values == compare.Comments_o.values:
            print('Type: ' + 'Old Post with Old Comments')
            nComments = pd.DataFrame()
        # 其他狀況，請釐清問題
        else:
            print('Type: ' + 'Uncertain condiction! Plz Check this website:' + '...')
            nComments = pd.DataFrame()        
        
        # post的資料直接concat在最下面，並duplicate掉，藉以更新互動摘要
        Posts = pd.concat([Posts, nPost], ignore_index=True)
        Posts = Posts.sort_values(by = 'Updatetime', ascending = False)
        Posts = Posts.drop_duplicates(subset ="Link",
                                      keep= 'first',
                                      inplace = False) 
        Comments = pd.concat([Comments, nComments], ignore_index=True, sort=False)
        print('Result: Succed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
    except:
        print('Result: Failed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
Posts.to_csv('C:/Users/TL_Yu/Desktop/Posts.csv',index = False)
Comments.to_csv('C:/Users/TL_Yu/Desktop/Comments.csv',index = False)

Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2439260866113541/
Type: Old Post with New Comments
Result: Succed!
Time Log: 2019-05-15 22:39:22
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2442288712477423/
Type: New Post
Result: Succed!
Time Log: 2019-05-15 22:39:39
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2449437375095890/
Type: New Post
Result: Succed!
Time Log: 2019-05-15 22:39:47
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2452941871412107/
Type: New Post
Result: Succed!
Time Log: 2019-05-15 22:40:22
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2451632571543037/
Type: New Post
Result: Succed!
Time Log: 201

Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:45:59
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/posts/2424456250927336
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:46:03
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/posts/2418299714876323
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:46:07
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/posts/2422602024446092
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:46:10
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/posts/2422597434446551
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:46:14
----------------------------------------------------------


Dealing with: https:

Type: Uncertain condiction! Plz Check this website:...
Result: Succed!
Time Log: 2019-05-15 22:48:32
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2392656157440679/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:48:36
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/videos/347523512532780/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:48:40
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2383803504992611/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:48:44
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2383802908326004/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:48:4

In [23]:
gc.collect()

0

In [22]:
datetime.datetime.now().strftime("%Y-%m-%d")

'2019-05-15'

In [23]:
expand('https://www.facebook.com/Aptg.tw/videos/296813794551595/')

In [ ]:
# 嘗試找文章的ID
# 文章的發布時間
# 順著ID連進去這篇文章
# 爬下文章的按讚數、留言數、分享數..等資訊
# 多少個朋友
# 直播影片數量
# 從留言的頭像可以找到發文者的ID


# 每周台灣之星與競業各自再努力些什麼，可以做為例行性的報告
#[Python] [網路爬蟲] 用 selenium 不用登入就能爬過 Facebook
#https://skylinelimit.blogspot.com/2018/09/python-selenium-facebook.html

In [18]:
# 原本想要登入的方式獲取連結，但FB會偵測異常行為，暫時先不登入
# 關閉Chrome的「通知」提醒
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")

driver = webdriver.Chrome(options=chrome_options)
url = 'https://www.facebook.com/'
driver.get(url)
time.sleep(3)

# 啟動瀏覽器並登入Facebook
username = driver.find_element_by_id('email')
username.send_keys('61034b001@gms.ndhu.edu.tw')
passwd=driver.find_element_by_id('pass')
passwd.send_keys('19920309')
button=driver.find_element_by_id('loginbutton')
button.click()

# Crawl2.0

In [2]:
import pandas as pd
import re, time, requests, datetime, gc
from selenium import webdriver
from bs4 import BeautifulSoup

In [3]:
driver = webdriver.Chrome()
driver.get('https://www.facebook.com/')
# print(driver.get_cookies())
driver.delete_all_cookies()
driver.add_cookie({'domain': '.facebook.com', 'expiry': 1558752322, 'httpOnly': False, 'name': 'locale', 'path': '/', 'secure': True, 'value': 'en_US'})

In [4]:
def CheckPosts(url, n):
    df = pd.DataFrame(columns = ['Link','Time','Comments']) 
    driver.get(url)
    for i in range(n):
        try:
            driver.find_element_by_css_selector('a.pam.uiBoxLightblue.uiMorePagerPrimary').click() # 加載更多貼文的按鈕
        except:
            time.sleep(2)
    # 這裡會跳出要我們登入的大畫面，找到「稍後再說」的按鈕並點擊
    driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
    soup = BeautifulSoup(driver.page_source)
    for i in soup.find_all('div', {'class':'_5pcr userContentWrapper'}):
        Time = i.find('abbr').attrs['title']
        Link = 'https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0]
        try:
            Comments = i.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        except:
            Comments = '0'
        df = pd.concat([df, pd.DataFrame(data = [{'Link':Link,
                                                  'Time':Time,
                                                  'Comments':Comments}],
                                         columns = ['Link','Time','Comments'])],
                      ignore_index = True)
    return df

In [5]:
def ToInt(x):
    if 'K' in x:
        Int = int(float(x.split('K')[0])*1000)
    else:
        Int = int(x)
    return Int

In [6]:
url = 'https://www.facebook.com/tstartel/'
CheckPosts = CheckPosts(url = url, n = 3)
CheckPosts['Comments'] = CheckPosts.Comments.apply(ToInt)
CheckPosts

,Link,Time,Comments
0,https://www.facebook.com/tstartel/photos/a.413...,"5/17/19, 9:00 PM",0
1,https://www.facebook.com/tstartel/photos/a.854...,"5/17/19, 5:10 AM",19
2,https://www.facebook.com/tstartel/photos/a.413...,"5/17/19, 1:00 AM",44
3,https://www.facebook.com/tstartel/posts/327684...,"5/16/19, 9:00 PM",14
4,https://www.facebook.com/tstartel/posts/327440...,"5/15/19, 10:11 PM",1200
5,https://www.facebook.com/tstartel/videos/32195...,"5/14/19, 9:00 PM",136
6,https://www.facebook.com/tstartel/posts/326935...,"5/13/19, 10:17 PM",48
7,https://www.facebook.com/tstartel/posts/326714...,"5/13/19, 1:38 AM",368
8,https://www.facebook.com/tstartel/photos/a.413...,"5/12/19, 6:20 AM",54
9,https://www.facebook.com/tstartel/photos/a.413...,"5/11/19, 9:00 PM",31


# 載入並比對先前資料
如果每次都把所有貼文抓下來，無疑會浪費程式效能，在這裡我將載入先前的資料，判斷Wall上的貼文屬於「新貼文」、「舊貼文但有新留言」、「舊貼文也無新留言」，後續再針對不同類型的貼文採取不同的動作。

In [7]:
#Posts = pd.DataFrame(columns=['Link','Comments'])
Posts = pd.read_csv('C:/Users/TL_Yu/Desktop/Posts.csv')
Posts = Posts.loc[:,['Link','Comments']]
Posts['Comments'] = Posts.Comments.apply(ToInt)
Posts

,Link,Comments
0,https://www.facebook.com/Aptg.tw/photos/a.1653...,2
1,https://www.facebook.com/Aptg.tw/photos/a.1653...,247
2,https://www.facebook.com/Aptg.tw/photos/a.1653...,1000
3,https://www.facebook.com/Aptg.tw/photos/a.1653...,14
4,https://www.facebook.com/Aptg.tw/posts/2367795...,8
5,https://www.facebook.com/Aptg.tw/posts/2368102...,5
6,https://www.facebook.com/Aptg.tw/posts/2369193...,1
7,https://www.facebook.com/Aptg.tw/posts/2369654...,1
8,https://www.facebook.com/Aptg.tw/photos/a.1653...,1
9,https://www.facebook.com/Aptg.tw/posts/2371062...,2


In [8]:
List = pd.merge(left=CheckPosts, right=Posts, how='left', on='Link', suffixes=('_c', '_p'), indicator=True)
ListA = List.loc[List._merge == 'left_only', :] # 新貼文
ListB = List.loc[List.Comments_c > List.Comments_p, :] # 舊貼文但有新留言
CarwlList = pd.concat([ListA, ListB], ignore_index=True)
CarwlList

,Link,Time,Comments_c,Comments_p,_merge
0,https://www.facebook.com/tstartel/photos/a.413...,"5/17/19, 9:00 PM",0,NaN,left_only
1,https://www.facebook.com/tstartel/photos/a.854...,"5/17/19, 5:10 AM",19,NaN,left_only
2,https://www.facebook.com/tstartel/photos/a.413...,"5/17/19, 1:00 AM",44,NaN,left_only
3,https://www.facebook.com/tstartel/posts/327684...,"5/16/19, 9:00 PM",14,NaN,left_only
4,https://www.facebook.com/tstartel/posts/327440...,"5/15/19, 10:11 PM",1200,NaN,left_only
5,https://www.facebook.com/tstartel/videos/32195...,"5/14/19, 9:00 PM",136,62.0,both
6,https://www.facebook.com/tstartel/posts/326935...,"5/13/19, 10:17 PM",48,36.0,both
7,https://www.facebook.com/tstartel/posts/326714...,"5/13/19, 1:38 AM",368,335.0,both
8,https://www.facebook.com/tstartel/photos/a.413...,"5/12/19, 6:20 AM",54,42.0,both
9,https://www.facebook.com/tstartel/photos/a.854...,"5/10/19, 2:28 AM",135,131.0,both


In [9]:
for i in CarwlList.Link:
    print(i)

https://www.facebook.com/tstartel/photos/a.413576055335114/3277738568918834/
https://www.facebook.com/tstartel/photos/a.854369967922385/3277739692252055/
https://www.facebook.com/tstartel/photos/a.413576055335114/3277132165646141/
https://www.facebook.com/tstartel/posts/3276845979008093
https://www.facebook.com/tstartel/posts/3274403725918985
https://www.facebook.com/tstartel/videos/321955001812559/
https://www.facebook.com/tstartel/posts/3269351643090860
https://www.facebook.com/tstartel/posts/3267141889978502
https://www.facebook.com/tstartel/photos/a.413576055335114/3259789660713725/
https://www.facebook.com/tstartel/photos/a.854369967922385/3259397344086290/
https://www.facebook.com/tstartel/photos/a.413576055335114/3246516712041020/


In [49]:
# 展開url中的所有留言 與 留言的留言
def Expand(url):
    driver.get(url)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    try:
        time.sleep(0.5)
        driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
        time.sleep(0.5)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        driver.find_element_by_id('expanding_cta_close_button').click()
        time.sleep(0.5)
        driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2ViewOptionsSelector/link"]').click()
        time.sleep(0.5)
        driver.find_element_by_partial_link_text('All Comments').click()
    except:
        print('There is no Comment')
    time.sleep(1)
    # 偵測是否有「更多留言」（第一層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))>0:
        for i in driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            try:
                driver.find_element_by_id('expanding_cta_close_button').click()
            except:
                time.sleep(0.5)            
            try:
                i.click()
            except:
                time.sleep(0.5)   
    # 偵測是否有「更多留言的留言」（第二層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))>0:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        for i in driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            try:
                driver.find_element_by_id('expanding_cta_close_button').click()
            except:
                time.sleep(0.5)
            try:
                i.click()
            except:
                time.sleep(0.5)
    # 偵測是否有「更多留言的留言」（第二層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))>0:
        for i in driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            try:
                i.click()
            except:
                time.sleep(0.5)

In [91]:
Expand(CarwlList.Link[2])

In [155]:
# 文章內容與互動摘要
def CrawlPost(soup):
    # po文區塊
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # po文人資訊區塊
    PosterInfo = userContent.find('div', {'class':'l_c3pyo2v0u i_c3pynyi2f clearfix'})
    # 互動摘要區(讚、留言與分享)
    feedback = soup.find('form', {'class':'commentable_item collapsed_comments'})
    # 名稱
    Name = PosterInfo.find('img').attrs['aria-label']
    # ID
    ID = userContent.find('div', {'class':'_5pcp _5lel _2jyu _232_'}).attrs['id'].split(';')[0].split('feed_subtitle_')[-1]
    # 網址
    Link = driver.current_url
    # 發文時間
    try:
        Time = PosterInfo.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = PosterInfo.find('div', {'class':'_1atc fsm fwn fcg'}).text
    # 文章內容
    try:
        Content = userContent.find('div', {'class':'_5pbx userContent _3576'}).text
    except:
        Content = ""
    # Like
    try:
        Like = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_LIKE'}).find('a').attrs['aria-label'].split(' ',2)[0]
        Like = int(Like)
    except:
        Like = 0 
    # Love
    try:
        Love = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_LOVE'}).find('a').attrs['aria-label'].split(' ',2)[0]
        Love = int(Love)
    except:
        Love = 0 
    # Haha
    try:
        Haha = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_HAHA'}).find('a').attrs['aria-label'].split(' ',2)[0]
        Haha = int(Haha)
    except:
        Haha = 0 
    # Wow
    try:
        Wow = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_WOW'}).find('a').attrs['aria-label'].split(' ',2)[0]
        Wow = int(Wow)
    except:
        Wow = 0 
    # Sad
    try:
        Sad = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_SORRY'}).find('a').attrs['aria-label'].split(' ',2)[0]
        Sad = int(Sad)
    except:
        Sad = 0 
    # Angry
    try:
        Angry = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_ANGER'}).find('a').attrs['aria-label'].split(' ',2)[0]
        Angry = int(Angry)
    except:
        Angry = 0
    # 留言
    try:
        Comments = feedback.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        Comments = ToInt(Comments)
    except:
        Comments = 0 
    # 分享
    try:
        Share = feedback.find('span', {'class':'_355t _4vn2'}).text.split(' ',2)[0]
    except:
        Share = '0' 
    Updatetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    return pd.DataFrame(
        data = [{'Name':Name,
                 'ID':ID,
                 'Link':Link,
                 'Time':Time,
                 'Content':Content,
                 'Like':Like,
                 'Love':Love,
                 'Haha':Haha,
                 'Wow':Wow,
                 'Sad':Sad,
                 'Angry':Angry,
                 'Comments':Comments,
                 'Share':Share,
                 'Updatetime':Updatetime}],
        columns = ['Name', 'ID', 'Time', 'Content', 'Like', 'Love', 'Haha', 'Wow', 'Sad', 'Angry', 'Comments', 'Share', 'Link', 'Updatetime'])

In [162]:
soup = BeautifulSoup(driver.page_source)
# 貼文區
userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
# 貼文區 > po文人資訊區塊
PosterInfo = userContent.find('div', {'class':'l_c3pyo2v0u i_c3pynyi2f clearfix'})
# 貼文區 > 留言區
UFI2CommentsList = soup.find('div', {'data-testid':'UFI2CommentsList/root_depth_0'})

In [163]:
CrawlPost(soup)

,Name,ID,Time,Content,Like,Love,Haha,Wow,Sad,Angry,Comments,Share,Link,Updatetime
0,台灣之星,360044337354953,2019-05-17 01:00,雖然 #00C編 的戰場不在考場在辦公室🤐 但還是心繫努力準備考試的同學們 還有在背後默默支...,256,3,18,0,0,0,49,0,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:26


In [184]:
# 留言
def CrawlComment(soup):
    Comments = pd.DataFrame()
    # 回應貼文的留言
    for i in soup.select('div._7a8- > ul > li'):
        Time = i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content']
        Time = Time.split(', ',1)[1]
        Time = datetime.datetime.strptime(Time, '%b %d, %Y at %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
        try:
            Content = i.find('span', {'dir':'ltr'}).text
        except:
            Content = 'sticker'
        Comment = pd.DataFrame(data = [{'ID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                        'Name':i.find('img').attrs['alt'],
                                        'Time':Time,
                                        'Content':Content,
                                        'repID':userContent.find('div', {'class':'_5pcp _5lel _2jyu _232_'}).attrs['id'].split(';')[0].split('feed_subtitle_')[-1],
                                        'repNM':userContent.find('img').attrs['aria-label'],
                                        'Link':driver.current_url}],
                               columns = ['ID', 'Name', 'Time', 'Content', 'repID', 'repNM','Link'])
        Comments = pd.concat([Comments, Comment], ignore_index=True)
    # 回應留言的留言
    for i in soup.select('div._7a8- > ul > li'):
        repID = i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1]
        repNM = i.find('img').attrs['alt']
        for i in i.findAll('div', {'data-testid':'UFI2Comment/root_depth_1'}):
            Time = i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content']
            Time = Time.split(', ',1)[1]
            Time = datetime.datetime.strptime(Time, '%b %d, %Y at %I:%M %p')
            Time = Time.strftime("%Y-%m-%d %H:%M")
            try:
                Content = i.find('span', {'dir':'ltr'}).text
            except:
                Content = 'sticker'
            Comment = pd.DataFrame(data = [{'ID':i.find('a', {'class':' _3mf5 _3mg1'}).attrs['data-hovercard'].split('id=',2)[1],
                                            'Name':i.find('img').attrs['alt'],
                                            'Time':Time,
                                            'Content':Content,
                                            'repID':repID,
                                            'repNM':repNM,
                                            'Link':driver.current_url}],
                                   columns = ['ID', 'Name', 'Time', 'Content', 'repID', 'repNM', 'Link'])
            Comments = pd.concat([Comments, Comment], ignore_index=True)
    Comments['Updatetime'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    return Comments

In [185]:
soup = BeautifulSoup(driver.page_source)
# po文區塊
userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
# 用戶留言區
UFI2CommentsList = soup.find('div', {'data-testid':'UFI2CommentsList/root_depth_0'})
CrawlComment(soup)

,ID,Name,Time,Content,repID,repNM,Link,Updatetime
0,100003717090073,Leslie Han,2019-05-17 16:03,支,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
1,100003906602217,褚又源,2019-05-17 16:09,3年前的明天 一下就過了3年了,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
2,100000896492255,順仁陳,2019-05-18 09:46,要新手機又要不迷路……必需 要辦“吃到飽” ;-),360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
3,100008036701674,陳國進,2019-05-17 16:20,台灣之星寫作文#小編陪我們一起寫作文,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
4,360044337354953,台灣之星,2019-05-17 16:08,Leslie的國文老師覺得很欣慰😆,100003717090073,Leslie Han,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
5,360044337354953,台灣之星,2019-05-17 16:22,3年後的你有沒有一樣乖乖讀書呀~,100003906602217,褚又源,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
6,360044337354953,台灣之星,2019-05-18 10:02,是很常迷路的概念嗎...🤣🤣,100000896492255,順仁陳,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
7,360044337354953,台灣之星,2019-05-17 16:28,國進是寫作文高手嗎~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
8,100008036701674,陳國進,2019-05-17 21:01,台灣之星 不是哦～作文我才1~3分而已,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
9,360044337354953,台灣之星,2019-05-17 21:29,沒關係 我們有 #辜歌大神🤣🤣,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42


In [186]:
CrawlComment(soup)

,ID,Name,Time,Content,repID,repNM,Link,Updatetime
0,100003717090073,Leslie Han,2019-05-17 16:03,支,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
1,100003906602217,褚又源,2019-05-17 16:09,3年前的明天 一下就過了3年了,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
2,100000896492255,順仁陳,2019-05-18 09:46,要新手機又要不迷路……必需 要辦“吃到飽” ;-),360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
3,100008036701674,陳國進,2019-05-17 16:20,台灣之星寫作文#小編陪我們一起寫作文,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
4,360044337354953,台灣之星,2019-05-17 16:08,Leslie的國文老師覺得很欣慰😆,100003717090073,Leslie Han,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
5,360044337354953,台灣之星,2019-05-17 16:22,3年後的你有沒有一樣乖乖讀書呀~,100003906602217,褚又源,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
6,360044337354953,台灣之星,2019-05-18 10:02,是很常迷路的概念嗎...🤣🤣,100000896492255,順仁陳,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
7,360044337354953,台灣之星,2019-05-17 16:28,國進是寫作文高手嗎~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
8,100008036701674,陳國進,2019-05-17 21:01,台灣之星 不是哦～作文我才1~3分而已,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
9,360044337354953,台灣之星,2019-05-17 21:29,沒關係 我們有 #辜歌大神🤣🤣,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 20:42
